---
title: "30538 Final Project: Reproducible Research - Volunteerism, Engagement, and Polarization in the U.S."
author: "Andrew White, Charles Huang, Justine Silverstein" 
date: "December 7, 2024"
format: pdf
execute:
  echo: false
---



1. Data wrangling (25%)


In [ ]:
# General
import pandas as pd
import numpy as np

# Visualization 
import altair as alt

# Geospatial data handling
import geopandas as gpd

# shiny framework
from shiny import App, ui, render

#for spatial data
from us import states

# Background

This project began as a shared interest in trends behind volunteering rates in America, as two of our members (Justine and Charles) are AmeriCorps alumni.

For the past few years, concerns about the American public's increasing rates of isolation, decreasing lack of civic engagement and faith in institutions, and greater rates of political polarization have been prominent in the news and media. Our personal experiences with AmeriCorps and volunteering have taught us that volunteering can be effective at reducing isolation, increasing civic engagement/community awareness, and decreasing negative polarization towards "the other side". However, is volunteering a legitimate part of a public policy solution to these issues, or is it just a red herring?

Our research questions were: 
1. What is the current state of volunteerism, political engagement and polarization in America? 
2. What factors make people more likely to volunteer or be civically engaged?


# Data Importing/Cleaning

Our datasets for this project were:

1. AmeriCorps CEV (Civic Engagement and Volunteering Supplement) for 2021
2. U.S. Census Bureau Volunteering and Civic Life Supplement - September 2021
3. ANES (American National Election Studies) Time Series Data, 2020

#1 and #2 primarily contain respondent information about volunteering and measures of civic engagement, while #3 contains information on political affiliation and polarization.

We are importing the data from the AmeriCorps and ANES websites. Because the datasets are over 100 MB, we include a Google Drive link here:

https://drive.google.com/drive/folders/1PUTN2pyh78MLoK0RVtGnf1ZwiM1BAAuV?usp=sharing


In [ ]:
#Google Drive link for data: https://drive.google.com/drive/folders/1PUTN2pyh78MLoK0RVtGnf1ZwiM1BAAuV?usp=sharing


cev_2021_raw = pd.read_csv("/Users/charleshuang/Documents/GitHub/student30538/problem_sets/final_project/data/2021_CEV__Current_Population_Survey_Civic_Engagement_and_Volunteering_Supplement_20241031.csv", encoding = 'utf-8')

vcl_supplement_raw = pd.read_csv("/Users/charleshuang/Documents/GitHub/student30538/problem_sets/final_project/data/sep21pub.csv")

polarization = pd.read_csv("/Users/charleshuang/Documents/GitHub/student30538/problem_sets/final_project/data/anes_timeseries_2020_csv_20220210.csv")

# Data cleaning - CEV/VCL data 

As there are over 400 variables in the CEV and VCL data, here are the most relevant variables we focused on:

Frequency and Type of Volunteering:
PES16: Did the respondent spend any time volunteering for any organization in the past 12 months?
PES16D: Frequency of volunteering (e.g., basically every day, a few times a week).
PTS16E: Approximate hours spent volunteering.

Political Engagement:
PES2: How often the respondent discussed political, societal, or local issues with friends or family.
PES5: How often these discussions occurred with neighbors.
PES13: Contact or visits to a public official to express opinions.
PES14: Boycotting or buying products based on political values or business practices.

Civic Participation and Group Membership:
PES15: Belonging to groups, organizations, or associations in the past 12 months.
Neighbor and Community Interaction:
PES7: Participation in activities to improve their neighborhood or community.
Voting Behavior:
PES11: Whether the respondent voted in the last local elections.

Social Media and News Consumption:
PES9: Posting views about political, societal, or local issues on the internet or social media.
PES10: Frequency of consuming news related to political or societal issues.

Basic Demographics:
Age: PRTAGE (Person's age)
Gender: PESEX (Sex of the respondent)
Race/Ethnicity: PTDTRACE (Detailed race and Hispanic origin)
Marital Status: PEMARITL (Marital status of the respondent)
Household Composition: HRNUMHOU (Number of persons in the household)

Potential Confounding Variables
Income: HEFAMINC (Household family income level)
Education: PEEDUCA (Highest level of school completed)
Urban/Rural Status: GTMETSTA (Metropolitan or non-metropolitan status)
Community Involvement: PES7 (Participation in neighborhood or community activities)
Social Media Use: PES9 (Posting views about political, societal, or local issues on the internet or social media)


In [ ]:
vcl_supplement_raw.columns = vcl_supplement_raw.columns.str.lower()

# I found out the data type is int64 for one df and object for the other

cev_2021_raw = cev_2021_raw.astype(str)
vcl_supplement_raw = vcl_supplement_raw.astype(str)

# This finds all the variables in common between the two for a merge
common_keys = list(set(vcl_supplement_raw.columns).intersection(
    set(cev_2021_raw.columns)))


cev_all_2021 = pd.merge(cev_2021_raw, vcl_supplement_raw,
                        on=common_keys, how="outer")

cev_all_2021.head(5)

# Note: Each row is a person- multiple people in the same household can have same household ID, so should not filter by unique

# We can use a config.py file to keep this readable. Full details are in the config.py file

In [ ]:
from config import selected_variables, rename_mapping

# selected_variables = ['hrhhid', 'hrhhid2', etc.]
# This chooses the variables we want from the merged raw data

# rename_mapping = "hrhhid": "Household_ID", "hrhhid2": "Household_ID_2", etc.
# Renaming the variables for clarity


cev_all_2021_filter = cev_all_2021[selected_variables].copy()
#copy avoids potentially modifying original dataframe

cev_all_2021_filter.rename(columns=rename_mapping, inplace=True)

#Debugging: removing duplicate column(s)
cev_all_2021_filter = cev_all_2021_filter.loc[:, ~cev_all_2021_filter.columns.duplicated()]

One data cleaning issue we encountered with the CEV/VCL data: the data is a mix of numeric code and qualitative input. We can create a mapping function to swap the numeric codes with qualitative input, but the existing qualitative input is outside of the data dictionary, so it won't get picked up by any mapping functions and will be transmuted into NaN data. We made a function that identifies all the values in the data that aren't picked up by our data dictionaries- this function is located in our config.py file.

For example, if there are some entries in a column that are already coded "Yes" or "No" in addition to "-1", "1", "2", etc. our existing mapping won't account for them and will turn them into NANs. We want to catch those and account for them.


In [ ]:
# Attribution: Asked ChatGPT "why isn't config.py updating when I add dictionaries to it; ChatGPT suggested using importlib reload"
# Import and reload config
from pathlib import Path
import os
from us import states
from config import add_engagement_score
from config import *
import importlib
import config
importlib.reload(config)


# Replace FIPS codes with states

fips_to_state = {int(state.fips): state.abbr for state in states.STATES}

cev_all_2021_filter['US State'] = pd.to_numeric(
    cev_all_2021_filter['US State'], errors='coerce')

cev_all_2021_filter['US State'] = cev_all_2021_filter['US State'].map(
    fips_to_state)

# Replace "Volunteered Past Year" data (pes16)
cev_all_2021_filter['Volunteered_Past_Year'] = cev_all_2021_filter['Volunteered_Past_Year'].map(
    pes16_dict)

# Replace "Volunteering Frequency" (pes16d)
cev_all_2021_filter['Volunteering_Frequency'] = cev_all_2021_filter['Volunteering_Frequency'].map(
    pes16d_dict)

# Replace "Hours Spent Volunteering" (pts16e)
cev_all_2021_filter['Hours_Spent_Volunteering'] = cev_all_2021_filter['Hours_Spent_Volunteering'].map(
    pts16e_dict)

# Replace "Discussed Issues with Friends/Family" data (pes2)
cev_all_2021_filter['Discussed_Issues_With_Friends_Family'] = cev_all_2021_filter['Discussed_Issues_With_Friends_Family'].map(
    pes2_dict)

# Replace "Discussed Issues with Neighbors" data (pes5)
cev_all_2021_filter['Discussed_Issues_With_Neighbors'] = cev_all_2021_filter['Discussed_Issues_With_Neighbors'].map(
    pes5_dict)

# Replace "Contacted Public Official" data (pes13)
cev_all_2021_filter['Contacted_Public_Official'] = cev_all_2021_filter['Contacted_Public_Official'].map(
    pes13_dict)

# Replace "Boycott Based on Values" data (pes14)
cev_all_2021_filter['Boycott_Based_On_Values'] = cev_all_2021_filter['Boycott_Based_On_Values'].map(
    pes14_dict)

# Replace "Belonged to Groups" data (pes15)
cev_all_2021_filter['Belonged_To_Groups'] = cev_all_2021_filter['Belonged_To_Groups'].map(
    pes15_dict)


# Replace "Community Improvement Activities" data (pes7)
cev_all_2021_filter['Community_Improvement_Activities'] = cev_all_2021_filter['Community_Improvement_Activities'].map(
    pes7_dict)

# Replace "Voted in Local Election" data (pes11)
cev_all_2021_filter['Voted_In_Local_Election'] = cev_all_2021_filter['Voted_In_Local_Election'].map(
    pes11_dict)

# Replace "Posted Views on Social Media" data (pes9)
cev_all_2021_filter['Posted_Views_On_Social_Media'] = cev_all_2021_filter['Posted_Views_On_Social_Media'].map(
    pes9_dict)

# Replace "Frequency of News Consumption" data (pes10)
cev_all_2021_filter['Frequency_Of_News_Consumption'] = cev_all_2021_filter['Frequency_Of_News_Consumption'].map(
    pes10_dict)

# Replace "Age" data (prtage)
cev_all_2021_filter['Age'] = cev_all_2021_filter['Age'].map(prtage_dict)

# Replace "Gender" data (pesex)
cev_all_2021_filter['Gender'] = cev_all_2021_filter['Gender'].map(pesex_dict)

# Replace "Race/Ethnicity" data (ptdtrace)
cev_all_2021_filter['Race_Ethnicity'] = cev_all_2021_filter['Race_Ethnicity'].map(
    ptdtrace_dict)

# Replace "Marital Status" data (pemaritl)
cev_all_2021_filter['Marital_Status'] = cev_all_2021_filter['Marital_Status'].map(
    pemaritl_dict)

# Replace "Household Size" data (hrnumhou)
cev_all_2021_filter['Household_Size'] = cev_all_2021_filter['Household_Size'].map(
    hrnumhou_dict)

# Replace "Family Income Level" data (hefaminc)
cev_all_2021_filter['Family_Income_Level'] = cev_all_2021_filter['Family_Income_Level'].map(
    hefaminc_dict)

# Replace "Education Level" data (peeduca_dict)
cev_all_2021_filter['Education_Level'] = cev_all_2021_filter['Education_Level'].map(
    peeduca_dict)

# Replace "Urban Rural Status" data (gtmetsta_dict)
cev_all_2021_filter['Urban_Rural_Status'] = cev_all_2021_filter['Urban_Rural_Status'].map(
    gtmetsta_dict)

# CEV/VCL Data - Measuring Political Engagement

As part of our analysis, we gauged volunteerism by referring to the question “Did you volunteer in the last 12 months”? However, there isn’t a single “civic/political engagement” question in the CEV/VCL data, but rather several different questions that are related.
We chose five of the most relevant questions and weighted each based on their level of effort:


1. “How frequently do you talk to a family member/neighbor about politics?” (15%)
2. “How frequently do you post political views on social media?” (15%)
3. “How frequently do you consume political news/media?” (10%)
4. “Did you contact an elected official to express your opinion in the last 12 months?” (30%) 
5. “Did you boycott a company based on their values in the last 12 months?” (30%)


This generated a score from 0 - 100 that we could use as a (imperfect) proxy for political engagement. We mutated a new variable, political_engagement_score, to measure this and added it to our dataset.


(Important caveat: For the political engagement questions, less than 20% of respondents answered three or more of the selected questions. To ensure meaningful data, we excluded all respondents who did not meet this threshold. While this approach improves the consistency of the dataset, we should be aware of potential selection bias.)

# ANES data cleaning

For the ANES data, we create


In [ ]:
#list 1
subset_list_1 = [column for column in polarization if "V2010" in column]

#list 2
subset_list_2 = [column for column in polarization if "V2012" in column]

#put the lists together
#make a loop 

#empty receiver
subset_list = []

#loop 1
for value in subset_list_1:
        subset_list.append(value)

#loop 2
for value in subset_list_2:
        subset_list.append(value)                    

#subset 
sub_polarization = polarization.filter(items= subset_list)

#add a column that just equals 1 to use for tracking the number
#of entries
sub_polarization["Observations"] = 1

In [ ]:
##Note: The below debugging code was suggested by ChatGPT to deal with some column value errors caused by political_engagement_score mutation. We weren't sure how to fix the errors without taking out the debugging code.

# Debugging code - Before engagement score calculation
print("Columns before adding engagement score:")
print(cev_all_2021_filter.columns.tolist())
print("\nShape before:", cev_all_2021_filter.shape)


# Mutate the engagement score variable we came up with in config.py
cev_all_2021_filter = add_engagement_score(cev_all_2021_filter)

# More debugging code
print("\nColumns after adding engagement score:")
print(cev_all_2021_filter.columns.tolist())
print("\nShape after:", cev_all_2021_filter.shape)

# Before saving
print("\nVerifying engagement score columns exist:")
print("'political_engagement_score' exists:",
      'political_engagement_score' in cev_all_2021_filter.columns)
print("'engagement_level' exists:",
      'engagement_level' in cev_all_2021_filter.columns)

In [ ]:
#confirm this code exports the data into the 'data' folder into the repo


# Lastly, export the cleaned data as a csv for plotting purposes
shiny_data_path = Path("shiny-app/basic-app/data")

# Create directory if it doesn't exist
shiny_data_path.mkdir(parents=True, exist_ok=True)

# Save with explicit column list to ensure all columns are included (debugging)
columns_to_save = cev_all_2021_filter.columns.tolist()
cev_all_2021_filter.to_csv(
    shiny_data_path / "cev_2021_cleaned.csv",
    columns=columns_to_save,
    index=False
)

# Verify saved file - more debugging
verification_df = pd.read_csv(shiny_data_path / "cev_2021_cleaned.csv")
print("\nColumns in saved CSV:")
print(verification_df.columns.tolist())

print(f"Dataset saved to: {shiny_data_path / 'cev_2021_cleaned.csv'}")

# Data Cleaning - ANES Data

As with the CEV/VCL data, our goal was to subset the data so that it only contains relevant variables. We accomplish this by making two lists:

List 1: This list is designed to capture variables covering geographic information (V201011, V201013a, V201013b, V201014a, V201014b)

List 2: This list is designed to capture variables covering information about assessments of political positioning (i.e. left, right, center)

In [ ]:
#list 1
subset_list_1 = [column for column in polarization if "V2010" in column]

#list 2
subset_list_2 = [column for column in polarization if "V2012" in column]

#put the lists together
#make a loop 

#empty receiver
subset_list = []

#loop 1
for value in subset_list_1:
        subset_list.append(value)

#loop 2
for value in subset_list_2:
        subset_list.append(value)                    

#subset 
sub_polarization = polarization.filter(items= subset_list)

#add a column that just equals 1 to use for tracking the number
#of entries
sub_polarization["Observations"] = 1

# ANES - More Data Cleaning

Analyzing Question V201200, which is a question asking:

"Where would you place yourself on this scale, or haven’t you
 thought much about this?
 Value Labels-9. Refused -8. Don’t know 
1. Extremely Liberal 
2. Liberal 
3. Slightly Liberal 
4. Moderate; middle of the road 
5. Slightly Conservative 
6. Conservative 
7. Extremely Conservative 
99. Haven’t thought much about this"


In [ ]:
#Make variables more clear

crosswalk_polar = pd.DataFrame({
    "Self_Rating_(V201200)":[-9, -8, 1, 2, 3, 4, 5, 6, 7, 99],
    "Ideology_(V201200)": [
        "Refused", "Don't Know", "Extremely Liberal",
        "Liberal", "Slightly Liberal", 
        "Moderate; middle of the road",
        "Slightly Conservative",
        "Conservative",
        "Extremely Conservative",
        "Haven’t thought much about this"
    ]
    })

#merge using crosswalk
sub_polarization = sub_polarization.merge(crosswalk_polar, left_on = "V201200", right_on = "Self_Rating_(V201200)")

We use this data to make a dataframe aggregated by state, and then we can show correlation between measure of polarity and the share of respondents in a state who did volunteer work.

 Note that we previously used two ANES variables as US State variables,  with only "US State 2" being used in analysis, purely because it has more in-universe entries. This appears to be partially due to respondent reactions to different questions, and partially due to information restrictions on the dataset. As such, "US State" is only used for CEV data. 


In [ ]:
#Replace FIPS codes with states
from us import states

fips_to_state = {int(state.fips): state.abbr for state in states.STATES}

sub_polarization["US State 2"] = pd.to_numeric(sub_polarization['V201014b'], errors='coerce')

sub_polarization['US State 2'] = sub_polarization['US State 2'].map(fips_to_state)


Then, make data using V201228, which asks:

"Generally speaking, do you usually think of yourself as [a
Democrat, a Republican / a Republican, a Democrat], an
independent, or what?"

-9. Refused
-8. Don’t know
-4. Technical error
0. No preference {VOL - video/phone only}
1. Democrat
2. Republican
3. Independent
5. Other party {SPECIFY}


In [ ]:
crosswalk_party_2 = pd.DataFrame({
    "Party_Numbers_(V201228)":[-9, -8, -4, 0, 1, 2, 3, 5],
    "Party_Affiliation_(V201228)":[
        "Refused", "Don't know", "Technical error", "No Preference", "Democrat",
        "Republican", "Independent", "Other party"
        ]})

sub_polarization = sub_polarization.merge(crosswalk_party_2, left_on = "V201228", right_on = "Party_Numbers_(V201228)")


# 2. Exploratory Analysis

# 2a. Measures of Polarization

We will use two measures of polarization from the ANES data; each provides some amount of information that can be interpreted to indicate polarization to a certain extent, though both have their drawbacks.

1. Share of Outliers - we create a series of functions that group respondents by party Democrats with conservative-leaning ideologies, and Republicans with liberal-leaning ones. 


In [ ]:
def find_share(df, state):
    """
    Processes a subset of survey data for a specific state to identify and count party-affiliated respondents and ideological outliers.

    Returns a subset of the input DataFrame, including new columns for:
        - 'Party_Count': Counts of individuals affiliated with major parties.
        - 'Outliers': Counts of ideological outliers within each party.
"""
    #subset by state
    sub = df[df["US State 2"] == state]
    Party_People = ["Democrat", "Republican"]
    Conservatives = ["Slightly conservative",
    "Conservative",
    "Extremely conservative"]
    Liberals = ["Extremely Liberal",
    "Liberal", "Slightly Liberal"]
    #count of partisans
    Party_Count = []
    #count of entries that are Dem but conservative or Repub but liberal
    Outlier_box = []
    #get Party_Count
    for index, entry in sub.iterrows():
            if entry["Party_Affiliation_(V201228)"] in Party_People:
                    Party_Count.append(1)
            else:
                    Party_Count.append(0)
    
    for index, entry in sub.iterrows():
            if (entry["Ideology_(V201200)"] in Conservatives) & (entry["Party_Affiliation_(V201228)"] == "Democratic"):
                    Outlier_box.append(1)
            elif (entry["Ideology_(V201200)"] in Liberals) & (entry["Party_Affiliation_(V201228)"] == "Republican"):
                    Outlier_box.append(1)
            else:
                    Outlier_box.append(0)
    
    sub["Party_Count"] = Party_Count
    sub["Outliers"] = Outlier_box
    return sub
        
Z = find_share(sub_polarization, "IL")                     

IL_Group = Z.groupby("Party_Affiliation_(V201228)")[["Outliers", "Party_Count"]].sum().reset_index()

IL_Group["Percent_Outliers"] =  IL_Group["Outliers"] / sum(IL_Group["Party_Count"])


In [ ]:
def seek_polar(df, state):
    '''
    Aggregates polarization data for a specific state, calculating the share of ideological outliers within party-affiliated populations.

    '''
    mod_df = find_share(df, state)
    mod_df = mod_df.groupby(
        "Party_Affiliation_(V201228)"
        )[["Outliers", "Party_Count"]].sum().reset_index()

    mod_df["Percent_Outliers"] =  mod_df["Outliers"] / sum(mod_df["Party_Count"])
    return mod_df

#Example with IL
seek_polar(sub_polarization, "IL")


In [ ]:
def find_share_state(df, state):
    '''
    Generates a modified subset of survey data for a given state, identifying partisans and ideological outliers.
'''
    #subset by state
    sub = df[df["US State 2"] == state]
    Party_People = ["Democrat", "Republican"]
    Conservatives = ["Slightly Conservative",
    "Conservative",
    "Extremely Conservative"]
    Liberals = ["Extremely Liberal",
    "Liberal", "Slightly Liberal"]
    #count of partisans
    Party_Count = []
    #count of entries that are Dem but conservative or Repub but liberal
    Outlier_box = []
    #get Party_Count
    for index, entry in sub.iterrows():
            if entry["Party_Affiliation_(V201228)"] in Party_People:
                    Party_Count.append(1)
            else:
                    Party_Count.append(0)
    
    for index, entry in sub.iterrows():
            if (entry["Ideology_(V201200)"] in Conservatives) & (entry["Party_Affiliation_(V201228)"] == "Democratic"):
                    Outlier_box.append(1)
            elif (entry["Ideology_(V201200)"] in Liberals) & (entry["Party_Affiliation_(V201228)"] == "Republican"):
                    Outlier_box.append(1)
            else:
                    Outlier_box.append(0)
    
    sub["Party_Count"] = Party_Count
    sub["Outliers"] = Outlier_box
    return sub


In [ ]:
def find_share_nation(df):
    '''
    Processes survey data at the national level to identify party-affiliated respondents and ideological outliers.
'''
    Party_People = ["Democrat", "Republican"]
    Conservatives = ["Slightly Conservative",
    "Conservative",
    "Extremely Conservative"]
    Liberals = ["Extremely Liberal",
    "Liberal", "Slightly Liberal"]
    #count of partisans
    Party_Count = []
    #count of entries that are Dem but conservative or Repub but liberal
    Outlier_box = []
    #get Party_Count
    for index, entry in df.iterrows():
            if entry["Party_Affiliation_(V201228)"] in Party_People:
                    Party_Count.append(1)
            else:
                    Party_Count.append(0)
    #get outlier count
    for index, entry in df.iterrows():
            if (entry["Ideology_(V201200)"] in Conservatives) & (entry["Party_Affiliation_(V201228)"] == "Democratic"):
                    Outlier_box.append(1)
            elif (entry["Ideology_(V201200)"] in Liberals) & (entry["Party_Affiliation_(V201228)"] == "Republican"):
                    Outlier_box.append(1)
            else:
                    Outlier_box.append(0)
    
    df["Party_Count"] = Party_Count
    df["Outliers"] = Outlier_box
    return df

In [ ]:
def polar_table(df):
    '''
    Generates a summary table of polarization data, grouping by state to calculate the share of ideological outliers.
    '''
    mod_df = find_share_nation(df)
    #make a grouped df that takes sum of each states' outliers and 
    #people who identify with a party
    mod_df = mod_df.groupby("US State 2")[
        ["Outliers", "Party_Count"]
        ].sum().reset_index()
    mod_df["Percent_Outliers"] =  mod_df["Outliers"] / sum(mod_df["Party_Count"])
    return mod_df


In [ ]:
polar_by_party = polar_table(sub_polarization)

#graphing percentages
percent_graph_outliers = alt.Chart(polar_by_party).mark_bar().encode(
    alt.X("US State 2", title = "State"),
    alt.Y("Percent_Outliers", title = "Outlier Share"),
    alt.Color("US State 2")
)

#graphing sums
sum_graph_outliers = alt.Chart(polar_by_party).mark_bar().encode(
    alt.X("US State 2", title = "State"),
    alt.Y("Outliers", title = "Outlier Sum"),
    alt.Color("US State 2")
)

In a paper on quantifying polarization written by Aaron Bramson et al (https://inferenceproject.yale.edu/sites/default/files/688938.pdf), the authors examine a range of polarization indicators. A relatively simple (and in some ways problematic) measurement is called spread, or dispersion- essentially the gap between the most extreme political positions.

In the paper, Bramson et al. explain: "Polarization in the sense of spread can be measured as the value of the agent with the highest belief value minus the value of the agent with the lowest belief value (sometimes called the ‘range’ of the data).""

We (imperfectly) approximate this using two more variables: V201206 and V201207. These ask respondents to position political parties on the political spectrum. We can select the most ideologically distant nodes on the personal ideology scale (extremely liberal and extremely conservative) and capture how far apart their conceptions of each party are,  on average, and then disaggregate by state.

We will assign the different ideological positions to different points on a spectrum, namely:
-3, -2, and -1 are "Extremely Liberal", 
"Liberal",  and "Slightly Liberal"; 

and:
0 is "Moderate; middle of the road"; 

finally,
1, 2, and 3 are "Slightly conservative",
"Conservative" and "Extremely conservative.""

We'll then compare average positions by state. For example, if the average 
extremely liberal respondent in Texas places Democrats at at -1 (slightly liberal) and the average for the extremely conservative respondents in -3 (extremely liberal), then the distance between the two is 4, meaning Texas would have a spread of 4 for this question.

We create the crosswalks V201206 and V201207 -

In [ ]:
crosswalk_V201206 = pd.DataFrame({
    #the dataset's scaled
    "Dem_Num_(V201206)":[-9, -8, 1, 2, 3, 4, 5, 6, 7],
    #the dataset's ideology
    "Dem_Ideology_(V201206)": [
        "Refused", "Don't Know", "Extremely Liberal",
        "Liberal", "Slightly Liberal", 
        "Moderate; middle of the road",
        "Slightly Conservative",
        "Conservative",
        "Extremely Conservative"
    ],
    #new values for comparison
    "Dem_Positioning(V201206)":[0, 0, -3, -2, -1, 0, 1, 2, 3]
    })

#They're exactly the same
crosswalk_V201207 = pd.DataFrame({
    #the dataset's scale
    "Repub_Num_(V201207)":[-9, -8, 1, 2, 3, 4, 5, 6, 7],
    #the dataset's ideology
    "Repub_Ideology_(V201207)": [
        "Refused", "Don't Know", "Extremely Liberal",
        "Liberal", "Slightly Liberal", 
        "Moderate; middle of the road",
        "Slightly Conservative",
        "Conservative",
        "Extremely Conservative"
    ],
    #new values for comparison
    "Repub_Positioning(V201207)":[0, 0, -3, -2, -1, 0, 1, 2, 3]
    })

#crosswalking V201206
sub_polarization = sub_polarization.merge(crosswalk_V201206, left_on = "V201206", right_on = "Dem_Num_(V201206)")

#crosswalking V201207
sub_polarization = sub_polarization.merge(crosswalk_V201207, left_on = "V201207", right_on = "Repub_Num_(V201207)")

#drop Dem_Num and Repub_Num for clarity (we'll be using the reassigned numbers)
sub_polarization = sub_polarization.drop(["Dem_Num_(V201206)", "Repub_Num_(V201207)"], axis = 1)

We use the Positioning columns to compare the average party position 
selections between the 2 extremes. 

Step 1: First grouping

In [ ]:
#filter out all self-identifiers that aren't "Extremely Liberal"
#or Extremely Conservative
ex_polarization = sub_polarization[
    (sub_polarization["Ideology_(V201200)"] == "Extremely Conservative") | 
    (sub_polarization["Ideology_(V201200)"] == "Extremely Liberal")
    ]

#filter out entries of Party Ideology Position that are "Don't Know" and "Refused"
ex_polarization = ex_polarization[
    (sub_polarization["Dem_Ideology_(V201206)"] != "Don't Know") &
    (sub_polarization["Dem_Ideology_(V201206)"] != "Refused") &
    (sub_polarization["Repub_Ideology_(V201207)"] != "Don't Know") &
    (sub_polarization["Repub_Ideology_(V201207)"] != "Refused") 
    ]


#make column for Liberal (i.e. Left tail) and Conservative (i.e. Right tail) 
Extreme_L_C = [
    "Liberal" if row["Ideology_(V201200)"] == "Extremely Liberal" else "Conservative" 
    for index, row in ex_polarization.iterrows()
    ]

#save to dataframe
ex_polarization["Extreme_Position"] = Extreme_L_C

#groupby Extreme_Position and US State 2 and get average Dem and Repub positioning. Then save this to a df called position_groups
position_groups = ex_polarization.groupby(["US State 2", "Extreme_Position"])[["Dem_Positioning(V201206)", "Repub_Positioning(V201207)"]].mean().reset_index()

Step 2: We use the variable position_groups to create a dataframe that has 4 columns:
(1) The position Liberals give Democrats on the spectrum
(2) the position Conservatives give Democrats on the spectrum
(3) the position Liberals give Republicans on the spectrum
(4) the position Conservatives give Republicans on the spectrum


In [ ]:
#Attribution: Asked ChatGPT for help fixing my list comprehension. Also asked ChatGPT how to flatten a multi-index column, which recommended ##source: https://www.w3resource.com/pandas/dataframe/dataframe-pivot.php


#pivot on Extreme_Position
pivot_position = position_groups.pivot(
    index = "US State 2", 
    columns = "Extreme_Position", 
    values = ["Dem_Positioning(V201206)", "Repub_Positioning(V201207)"]
    )

#reset index to retrieve US State 2 variable from index
pivot_position = pivot_position.reset_index()

#Flatten multi-index by joining the upper and lower
#levels of the multi-indexed columns together
pivot_position.columns = ['_'.join(col).strip() for col in pivot_position.columns.values]

#rename flattened columns
pivot_position = pivot_position.rename(
        columns = {
            "US State 2_":"US State 2", 
            "Dem_Positioning(V201206)_Conservative":"Dem_Position_C",
            "Dem_Positioning(V201206)_Liberal":"Dem_Position_L",
            "Repub_Positioning(V201207)_Conservative":"Repub_Position_C",
            "Repub_Positioning(V201207)_Liberal":"Repub_Position_L"
        }
    )


Step 3: Now, we use those 4 columns to create the spread, meaning the absolute value of the difference betweeen:

(1) The position Liberals give Democrats on the spectrum and the position Conservatives give Democrats on the spectrum

(2) The position Liberals give Republicans on the spectrum and the position Conservatives give Republicans on the spectrum


In [ ]:
#Attribution: Used this article: https://www.w3resource.com/pandas/dataframe/dataframe-pivot.php
#Asked ChatGPT how to flatten a multi-index column

#capture spread variables
pivot_position["Spread_Dem"] = abs(
    pivot_position["Dem_Position_C"] - 
    pivot_position["Dem_Position_L"]
    )

pivot_position["Spread_Repub"] = abs(
    pivot_position["Repub_Position_C"] - 
    pivot_position["Repub_Position_L"]
    )

Step 4: Now, we graph those differences and interpret these spreads as an indicator of polarization. We acknowledge that this is too simple an analysis to account for the full complexity of this kind of measurement, as polarization involves not just distance between extremes, but also clustering around them. We also acknowledge that our political scale assumes a linear ideological spectrum, which isn't always the case.


In [ ]:
#Opinion on the position of the Democratic Party
Democratic_Party_Spread_Graph = alt.Chart(pivot_position).mark_bar().encode(
    alt.X("US State 2", title = "State"),
    alt.Y("Spread_Dem", 
    title = "Spread of Views"),
    alt.Color("US State 2")
).properties(title = alt.TitleParams("Spread of Views on Democratic Party Ideological Position"))

#Opinion on the position of the Republican Party
Republican_Party_Spread_Graph = alt.Chart(pivot_position).mark_bar().encode(
    alt.X("US State 2", title = "State"),
    alt.Y("Spread_Repub", 
    title = "Spread of Views"),
    alt.Color("US State 2")
).properties(title = alt.TitleParams("Spread of Views on Republican Party Ideological Position"))

Step 5: We merge the two datasets together on "US State 2". Note that this merged dataset could be misleading, because some of the merged variables only apply to the "extreme" respondents in each state, and so it's not representative of the entire state's respondents' positioning of different parties. We've dropped those variables from the merge to try to account for this.


In [ ]:
merged_polarization = sub_polarization.merge(pivot_position, left_on = "US State 2", right_on = "US State 2", how = "outer")

#drop variables that don't make sense when merged
merged_polarization = merged_polarization.drop([
    "Dem_Position_C", "Dem_Position_L",
    "Repub_Position_C", "Repub_Position_L"], 
    axis = 1
    )

Step 6: We merge the AmeriCorps CEV/VCL data with the spread data so far


In [ ]:
#remove 'Not in Universe', 'No Answer', 'Refused'
#from cev_all_2021_filter

Non_Number_List = [
    'Not in Universe', np.nan, "Do Not Know",
    'No Answer', 'Refused']

cev_all_2021_all_number = cev_all_2021_filter[
    ~cev_all_2021_filter["Hours_Spent_Volunteering"].isin(Non_Number_List)
    ]

#group volunteer rates by state
group_cev = cev_all_2021_all_number.groupby(
    "US State"
    )[["Hours_Spent_Volunteering", 
    "political_engagement_score"]].mean().reset_index()

main_group = group_cev.merge(
    pivot_position, left_on = "US State", right_on = "US State 2"
)

Step 7: We create a function for viewing Political Engagement alongside spread, one generating a table with 2 states for comparison, and another 2 
returning graphs


In [ ]:
#return table
def engagement_spread_table(df, state_1, state_2):
        sub = df[
            (df["US State"] == state_1) |
            (df["US State"] == state_2)]
        sub = sub.filter(["US State", "political_engagement_score", "Spread_Dem", "Spread_Repub"])
        return sub

#return graph with Democratic positioning 
def engagement_spread_graph_D(df, column):
        graph = alt.Chart(df).mark_bar().encode(
        alt.X("US State 2", title = "State", sort = "-y"),
        alt.Y("Spread_Dem", title = "Spread of Extreme Views"),
         alt.Color(
            column, 
            scale = alt.Scale(range = ["lightblue", "darkblue"])
            )
        ).properties(
            title = alt.TitleParams(
                "Spread of Views on Democratic Party Ideological Position"
                )
            )
        return graph

#return graph with Republican positioning 
def engagement_spread_graph_R(df, column):    
        graph = alt.Chart(df).mark_bar().encode(
        alt.X("US State 2", title = "State", sort = "-y"),
        alt.Y("Spread_Repub", title = "Spread of Extreme Views"),
        alt.Color(
            column, 
            scale = alt.Scale(range = ["lightblue", "darkblue"]),
            )
        ).properties(
            title = alt.TitleParams(
                "Spread of Views on Republican Party Ideological Position"
                )
            )
        return graph

Below is an example of using our function to generate a political engagement score for different states: 


In [ ]:
#sample static table with 
Sample_Table = engagement_spread_table(main_group, "IL", "OR")

print(Sample_Table)

And below are sample static graphs on Democratic and Republican positions:

In [ ]:
#using average political engagement
Dem_Graph_Spread_1 = engagement_spread_graph_D(main_group, "political_engagement_score")

#using average volunteer hours
Dem_Graph_Spread_2 = engagement_spread_graph_D(main_group, "Hours_Spent_Volunteering")

Dem_Graph_Spread_1.show()

Dem_Graph_Spread_2.show()

In [ ]:
#using average political engagement
Repub_Graph_Spread_1 = engagement_spread_graph_R(main_group, "political_engagement_score")

#using average volunteer hours
Repub_Graph_Spread_2 = engagement_spread_graph_R(main_group, "Hours_Spent_Volunteering")

Repub_Graph_Spread_1.show() 

Repub_Graph_Spread_2.show()

Below is a atatic graph of ideological position in the US nationally as of 2020:


In [ ]:
alt.data_transformers.enable("vegafusion")
#create list for graph to sort on 
sorting_list = [
        "Refused", "Don't Know", "Extremely Liberal",
        "Liberal", "Slightly Liberal", 
        "Moderate; middle of the road",
        "Slightly Conservative",
        "Conservative",
        "Extremely Conservative",
        "Haven’t thought much about this"
    ]

#filter out non-responses
global_non_response_list = [
    "Haven’t thought much about this",
    "Refused", "Don't Know"
    ]

local_filter = sub_polarization[
    ~sub_polarization["Ideology_(V201200)"].isin(global_non_response_list)
]

Ideological_Position_US = alt.Chart(local_filter).mark_bar().encode(
    alt.X(
        "Ideology_(V201200):N", 
        title = "Ideological Position",
        sort = sorting_list),
    alt.Y("Observations", title = "Number of Respondents"),
    alt.Color("Ideology_(V201200)", sort = sorting_list)
).properties(title = alt.TitleParams(
            f"Ideological Position in the U.S."
        ))

Ideological_Position_US.show()

Note: We removed non-responses, likely creating a bias in the relative
size of the remaining groups (it's difficult to predict the direction
of said bias, however.)

Additionally, below is a sample static graph of ideological position by state, using IL as an example:


In [ ]:
def ideology_by_state(df, state):
        #filter out non-responses
        non_response_list = [
            "Haven’t thought much about this", 
            "Refused", "Don't Know"
            ]
        sub = df[
            ~df["Ideology_(V201200)"].isin(non_response_list)
            ]
        #filter for the selected state
        sub = sub[sub["US State 2"] == state]
        #create list for graph to sort on 
        sorting_list = [
        "Extremely Liberal",
        "Liberal", "Slightly Liberal", 
        "Moderate; middle of the road",
        "Slightly Conservative",
        "Conservative",
        "Extremely Conservative"
        ]
        graph = alt.Chart(sub).mark_bar().encode(
            alt.X(
                "Ideology_(V201200):N", 
                title = "Ideological Position",
                sort = sorting_list
                ),
            alt.Y("count():Q", title = "Number of Respondents"),
            alt.Color("Ideology_(V201200):N", sort = sorting_list)
        ).properties(title = alt.TitleParams(
            f"Ideological Position in {state}"
        ))
        return graph

#Example for Illinois
Ideological_Position_IL = ideology_by_state(sub_polarization, "IL")

Ideological_Position_IL.show()

Lastly, we will use the merged AmeriCorps and ANES data to plot a correlation between volunteer hours and political engagement:


In [ ]:
Correlation_Graph = alt.Chart(main_group).mark_circle().encode(
    alt.X("Hours_Spent_Volunteering", title="Average Volunteer Hours"),
    alt.Y("political_engagement_score", title="Average Political Engagement Score").scale(
        domain=(
            main_group["political_engagement_score"].min() - 3,
            main_group["political_engagement_score"].max() + 3
        )
    ),
    alt.Color("US State 2")
).properties(title=alt.TitleParams(
    "Correlation between volunteer hours and political engagement"
))

Correlation_Graph.show()

This shows a noted positive correlation between volunteer hours and political engagement. However, as we'll see, this doesn't necessarily imply that increasing volunteering will lead to higher civic engagement.

4. Reproductibility (10%)
    * The project and files should be structured and documented so that the TAs can clone your repository and reproduce your results (see "Final Repository" below) by knitting your `.qmd` and, if needed, downloading the dataset(s) you use using the link provided in the `.qmd` comments
5. Git (10%)
    * You should submit your project as a Git repository.
    * Create multiple branches as you work for different pieces of the analysis. Branches may correspond to work done by different partners or to different features if you are working alone.
    * Your final repository should have one branch: `main`
    * We reserve the right to check the git commit history to ensure that all members have contributed to the project.
6. Extra credit: text processing (up to 10%)
    * Introduce some form of text analysis using natural language processing methods discussed in class.

## Writeup (15%)
* You will then spend *no more than 3 pages* writing up your project. 
* The primary purpose of this writeup is to inform us of what we are reading before we look at your code.
* You should describe your research question, then discuss the approach you took and the coding involved, including discussing any weaknesses or difficulties encountered. 
* Display your static plots, and briefly describe them and your Shiny app. Discuss the policy implications of your findings.
* Finish with a discussion of directions for future work. 
* The top of your writeup should include the names of all group members, their respective sections, and Github user names.

## Presentation (15%)
* On the day of the presentation, one of the group members will be *randomly selected* to give a *8-minute in-class presentation*. All group members must be present.
* Any group member who is not present will receive an automatic 0 for the presentation portion of the final project.
* The presentation will be of slides that largely mirror the structure of the writeup, but will be more focused on discussing the research question and results as opposed to explaining the details of the coding. 

# Final Repository
Your final repository must contain the following:

* Documentation and Meta-data
    * A `requirements.txt` file 
    * A `.gitignore` file that ignores unneeded files (e.g. `venv`) 
* Writeup: a user should be able to knit your `.qmd` file and re-generate the HTML version of your writeup
    * The `.qmd` file associated with your write-up
    * An HTML and PDF'd version of your writeup
    * A folder named `pictures` that contains the files for any pictures required to knit your writeup
* Data
    * A folder named `data` that contains the initial, unmodified dataframes you download and the final versions of the dataframe(s) you built.
    * If the dataset is greater than 100MB, it can hosted on Drive or Dropbox and the link should be provided in your .`qmd` file as a comment
* Shiny app
    * A folder named `shiny-app` that contains the code and any additional files needed to deploy your app
    * A user should be able to deploy your app directly from the command line within this folder


# Key Dates
* By November 1
    * Proposal submitted to Canvas quiz
    * (Optional) meeting with Professor Ganong, Professor Shi, or Head TA Ozzy Houck
    * Sign up for presentation slot
* December 2- December 5: in-class presentations
* December 7, 5PM: final repository submitted via Gradescope